## Setting up the model

#### Import libraries

In [1]:
import numpy as np
import tensorflow as tf
tf.__version__

'2.10.0'

#### Import the data

In [2]:
npz = np.load("Audiobooks_data_train.npz")

train_inputs = npz["inputs"].astype(float)
train_targets = npz["targets"].astype(int)

npz = np.load("Audiobooks_data_validation.npz")

validation_inputs = npz["inputs"].astype(float)
validation_targets = npz["targets"].astype(int)

npz = np.load("Audiobooks_data_test.npz")

test_inputs = npz["inputs"].astype(float)
test_targets = npz["targets"].astype(int)

print("training data size:", len(train_targets))
print("validation data size:", len(validation_targets))
print("test data size:", len(test_targets))

training data size: 3579
validation data size: 447
test data size: 448


Above, the inputs and the targets are in array form:

#### Set up the model

In [3]:
print(len(train_inputs[0]))

10


In [4]:
input_size = len(train_inputs[0])
output_size = 2 # we will get either a 0 or a 1
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation="tanh"), 
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"), 
                            tf.keras.layers.Dense(hidden_layer_size, activation="tanh"), 
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(output_size, activation="softmax") 
                            ])

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

batch_size = 50
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(train_inputs,train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks = [early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)

Epoch 1/100
72/72 - 1s - loss: 0.4306 - accuracy: 0.7645 - val_loss: 0.3315 - val_accuracy: 0.8345 - 1s/epoch - 14ms/step
Epoch 2/100
72/72 - 0s - loss: 0.3767 - accuracy: 0.7941 - val_loss: 0.3239 - val_accuracy: 0.8076 - 128ms/epoch - 2ms/step
Epoch 3/100
72/72 - 0s - loss: 0.3694 - accuracy: 0.7977 - val_loss: 0.3106 - val_accuracy: 0.8434 - 122ms/epoch - 2ms/step
Epoch 4/100
72/72 - 0s - loss: 0.3676 - accuracy: 0.7999 - val_loss: 0.3151 - val_accuracy: 0.8345 - 128ms/epoch - 2ms/step
Epoch 5/100
72/72 - 0s - loss: 0.3585 - accuracy: 0.7997 - val_loss: 0.3111 - val_accuracy: 0.8501 - 125ms/epoch - 2ms/step
Epoch 6/100
72/72 - 0s - loss: 0.3605 - accuracy: 0.7994 - val_loss: 0.3094 - val_accuracy: 0.8412 - 124ms/epoch - 2ms/step
Epoch 7/100
72/72 - 0s - loss: 0.3596 - accuracy: 0.7960 - val_loss: 0.3074 - val_accuracy: 0.8434 - 130ms/epoch - 2ms/step
Epoch 8/100
72/72 - 0s - loss: 0.3532 - accuracy: 0.8078 - val_loss: 0.3068 - val_accuracy: 0.8412 - 135ms/epoch - 2ms/step
Epoch 9/10

#### Test the model

In [5]:
test_estimations = model.predict_on_batch(test_inputs)
estimation_results = []

for estimate in test_estimations:
    if estimate[0] > estimate[1]:
        estimation_results.append(0)    
    else:
        estimation_results.append(1)

print("num. of estimations:",len(estimation_results))

for i in range(len(estimation_results)):
    print("Estimation: {}, Target: {}".format(estimation_results[i],test_targets[i]))

num. of estimations: 448
Estimation: 1, Target: 1
Estimation: 1, Target: 1
Estimation: 1, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 1
Estimation: 1, Target: 1
Estimation: 1, Target: 1
Estimation: 1, Target: 1
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 1
Estimation: 1, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 1
Estimation: 1, Target: 1
Estimation: 0, Target: 0
Estimation: 1, Target: 1
Estimation: 1, Target: 1
Estimation: 1, Target: 0
Estimation: 0, Target: 1
Estimation: 1, Target: 1
Estimation: 0, Target: 1
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 1, Target: 1
Estimation: 1, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 0, Target: 0
Estimation: 1, Target: 1
Estimation: 0, Target: 0
Estimation: 0, Target: 1
Estimation: 1, Target: 1
Estimation: 1, Target: 1
Estimation: 0, Target: 0
Estimation: 0, Target: 1
Estimation: 0, Target: 0


In [6]:
wrong_estimations = 0

for i in range(len(estimation_results)):
    if estimation_results[i] != test_targets[i]:
        wrong_estimations +=1

print("Model accuracy: {}".format(1 - wrong_estimations/len(estimation_results)))

Model accuracy: 0.7700892857142857


In [7]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.3926 - accuracy: 0.7701
